In [ ]:
#import required libraries
from azure.ml import MLClient
from azure.ml.entities import CommandJob, Code, PipelineJob, Dataset, InputDatasetEntry

In [ ]:
#Enter details of your AML workspace
subscription_id = 'b17253fa-f327-42d6-9686-f3e553e24763'
resource_group = 'testharnvir-local'
workspace = 'diab-ws01'

In [ ]:
#get a handle to the workspace
ml_client = MLClient(subscription_id, resource_group, workspace)

In [ ]:
#define the prep-job to run in the pipeline
prep_cmd = "pip freeze && echo ${{inputs.max_epocs}} && python prep.py --raw_data ${{inputs.raw_data}} --prep_data ${{outputs.prep_data}}"
prep_data = Dataset(
    local_path="./data"
)

pipeline_job_inputs = {'max_epocs_from_pipeline': '20', 'raw_data_from_pipeline': prep_data}
prep_job_inputs = {'max_epocs': '${{inputs.max_epocs_from_pipeline}}', 'raw_data': InputDatasetEntry(dataset=prep_data)}
prep_job_outputs = {'prep_data': None}

job1 = CommandJob(
    code=Code(local_path="./src/prep"), #local path where the code is stored
    command = prep_cmd,
    inputs=prep_job_inputs,
    outputs=prep_job_outputs,
    environment = "AzureML-sklearn-0.24-ubuntu18.04-py37-cuda11-gpu:9",
    compute = "cpu-cluster"
)

In [ ]:
# lets create the pipeline
pipeline_job = PipelineJob(
    description = 'nyc-taxi-pipeline-example',
    jobs= {'prep-job':job1},
    inputs= pipeline_job_inputs,
    outputs=prep_job_outputs,
    compute = "cpu-cluster"
    # compute={"target": "cpu-cluster"},
)

In [ ]:
#submit the pipeline job
returned_job = ml_client.jobs.create_or_update(pipeline_job)
#get a URL for the status of the job
returned_job.services["Studio"].endpoint